# Example of using LLM-based pipeline with augmentation for recommendation task


Run in google colaboratory

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import sys
import os

repo_path = '/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec'
sys.path.append(repo_path)

## Install requirements

In [ ]:
!pip install -q -r '{repo_path}/requirements/requirements.txt'

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 973.5/973.5 kB 10.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 308.5/308.5 kB 25.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 27.0/27.0 MB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.5/85.5 MB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.6/320.6 kB 40.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 45.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 18.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 26.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.8/122.8 kB 16.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 7.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 105.4/105.4 kB 16.5 

## Add config

In [ ]:
import os

config_dict = {
    "csv_args": {"delimiter": "\t"},
    "source_column": "item_id:token",
    "search_kwargs": {"k": 200},
    "data_path": os.path.join(repo_path, "datasets"),
    "load_col": {
        "inter": ["user_id", "item_id", "rating", "timestamp"],
        "item": ["item_id", "movie_title"],
    },
    "text_col": ["movie_title", "release_year", "class"],
    "MAX_ITEM_LIST_LENGTH": 10,
    "eval_args": {"split": {"LS": "valid_and_test"}, "order": "TO", "mode": "full"},
    "repeatable": True,
    "loss_type": "CE",
    "train_batch_size": 100,
    "eval_batch_size": 8,
    "valid_metric": "NDCG@10",
    "metrics": ["Recall", "NDCG"],
    "topk": [1, 5, 20],
    "train_neg_sample_args": None,
}

In [ ]:
assert config_dict['search_kwargs']['k'] >= max(config_dict['topk'])

## Get dataset and config

In [ ]:
from llm4rec.pipelines import RecBolePipelineRecommender
from llm4rec.dataset import RecboleSeqDataset
from llm4rec.trainer import PipelineTrainer
from recbole.data.utils import data_preparation
from recbole.config import Config
from recbole.model.abstract_recommender import AbstractRecommender
import os
import torch

model_cls = RecBolePipelineRecommender
dataset_name = 'ml-100k'

config = Config(model=model_cls, dataset=dataset_name,
            config_dict=config_dict)

dataset = RecboleSeqDataset(config)
train_data, _, eval_data = data_preparation(config, dataset)

## Create llm and embeddings instances for pipeline

In [ ]:
from dotenv import load_dotenv

path_to_openai_env = os.path.join(repo_path, "api_keys.env")
load_dotenv(path_to_openai_env)

In [ ]:
from langchain import HuggingFaceHub
from langchain_community.embeddings import HuggingFaceEmbeddings
import torch

llm = HuggingFaceHub(repo_id="mistralai/Mistral-7B-Instruct-v0.2", model_kwargs={"temperature":0.1, "max_new_tokens":512,
                                                                                  "return_full_text":False})
embedding_size = 384
embedding_fn = HuggingFaceEmbeddings(
                model_name="all-MiniLM-L6-v2", model_kwargs={"device":"cuda:0" if torch.cuda.is_available() else "cpu"})

/usr/local/lib/python3.10/dist-packages/langchain_core/_api/deprecation.py:119: LangChainDeprecationWarning: The class `HuggingFaceHub` was deprecated in LangChain 0.0.21 and will be removed in 0.3.0. Use HuggingFaceEndpoint instead.
  warn_deprecated(
/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.7k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

## Create item and user memory (for item info augmentation and for generation of user profile from train data)

In [ ]:
from llm4rec.memory import ItemMemory, UserMemory
from functools import partial
from langchain_community.document_loaders import WikipediaLoader

wiki_loader = partial(WikipediaLoader, load_max_docs=1)

item_filepath = "/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec/examples/develop/item_memory_summ.json"
user_filepath = "/content/drive/MyDrive/Colab Notebooks/thesis_work/llm-for-rec/examples/develop/user"

item_memory = ItemMemory(item_ids=dataset.item_id_token[1:],
                         title_col='movie_title',
                         dataset_info_map=dataset.item_token2attr,
                         load_filename=item_filepath,
                         summary_llm=llm,
                         augmentation_loader=wiki_loader)

user_memory = UserMemory(user_attributes=dataset.user_token2text,
                         short_term_limit=20, llm=llm,
                         embeddings=embedding_fn,
                         item_memory=item_memory,
                         emb_size=embedding_size,
                         load_filename=None,
                         train_dataset=train_data.dataset)

In [ ]:
item_memory.save(item_filepath)
user_memory.save(user_filepath)

## Define tasks in pipeline

In [ ]:
from llm4rec.tasks import ItemAugmentation, UserAugmentation, RetrievalRecommender, RankerRecommender

item_augm = ItemAugmentation(item_memory)
user_augm = UserAugmentation(user_memory)
retrieval = RetrievalRecommender(
                embeddings=embedding_fn,
                item2text=lambda x: item_memory.retrieve(x, retr_type=''),
                load_from_file=False,
                item_memory=item_memory,
                search_kwargs={'k':max(config['topk'])}
)
ranker = RankerRecommender(llm=llm)

## Create and evaluate pipeline

In [ ]:
model = RecBolePipelineRecommender(config=config, dataset=dataset,
                                   tasks=[item_augm, user_augm, retrieval, ranker],
                                   verbose=True)

Evaluate for one user

In [ ]:
for batched_data in eval_data:
    interaction, history_index, positive_u, positive_i = batched_data
    batch_size = len(interaction["user_id"])

    for inter_idx in range(batch_size):
        user_id = interaction[inter_idx]["user_id"]
        history_ids = interaction[inter_idx]["item_id_list"]
        history_length = min(
            config["MAX_ITEM_LIST_LENGTH"],
            interaction[inter_idx]["item_length"],
        )
        history_names = [eval_data.dataset.item_id2text(hist_id) for hist_id in history_ids[:history_length].tolist()]
        history_item_ids = eval_data.dataset.id2token("item_id", history_ids[:history_length])
        user_id_token  = eval_data.dataset.id2token('user_id', user_id)
        reco = model.run(user_profile=user_id_token, prev_interactions=history_item_ids, top_k=5)

        user_memory.update(user_id_token, {'item_id':history_item_ids[-1], 'rating':interaction[inter_idx]['rating'][-1]})
        print(reco)
        break
    break

Task 1 outputs:  {'172': "movie_title: Empire Strikes Back, The; release_year: 1980; class: Action Adventure Drama Romance Sci-Fi War; additional info: The Empire Strikes Back, released in 1980, is the second installment in the Star Wars saga, following the Rebel Alliance three years after the destruction of the Death Star. The Galactic Empire, led by Darth Vader, dispatches probe droids to locate the Rebels. One probe finds their base on the ice planet Hoth, and the Rebels are forced to evacuate when the Empire launches an attack using AT-AT walkers. During the chaos, Luke Skywalker is captured by a wampa but escapes using the Force. He is instructed by Obi-Wan Kenobi's spirit to train as a Jedi Knight on Dagobah under Yoda. Han Solo rescues Luke, and they go to Cloud City, run by Han's old friend Lando Calrissian, to repair their damaged ship. However, Vader tracks them down and captures Han, revealing he is Luke's father. The film faced production difficulties and was initially met 